# 07wk-1: (합성곱신경망) – CNN 자랑, CNN 핵심레이어

최규빈  
2025-04-16

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/07wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-wZj26ZHPIZ6kvcrPiL92q3&si=UF3qTgEXmUqnV4e->

# 2. Imports

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [2]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. CNN 자랑

## A. 성능좋음

*Fashion MNIST*

In [21]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)
train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
test_dataset = torch.utils.data.Subset(test_dataset, range(1000))
to_tensor = torchvision.transforms.ToTensor()
X = torch.stack([to_tensor(img) for img, lbl in train_dataset]).to("cuda:0")
y = torch.tensor([lbl for img, lbl in train_dataset])
y = torch.nn.functional.one_hot(y).float().to("cuda:0")
XX = torch.stack([to_tensor(img) for img, lbl in test_dataset]).to("cuda:0")
yy = torch.tensor([lbl for img, lbl in test_dataset])
yy = torch.nn.functional.one_hot(yy).float().to("cuda:0")

*발악수준으로 설계한 신경망*

In [31]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [32]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [33]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

In [34]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

*대충대충 설계한 합성곱신경망*

In [35]:
torch.manual_seed(0)
net = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [36]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [37]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

In [38]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

## B. 파라메터적음

In [39]:
net1 = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
)
net2 = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
)

In [40]:
net1_params = list(net1.parameters())
print(net1_params[0].shape)
print(net1_params[1].shape)
print(net1_params[2].shape)
print(net1_params[3].shape)

torch.Size([2048, 784])
torch.Size([2048])
torch.Size([10, 2048])
torch.Size([10])

In [41]:
2048*784 + 2048 + 10*2048 + 10 

In [42]:
net2_params = list(net2.parameters())
print(net2_params[0].shape)
print(net2_params[1].shape)
print(net2_params[2].shape)
print(net2_params[3].shape)

torch.Size([16, 1, 2, 2])
torch.Size([16])
torch.Size([10, 2704])
torch.Size([10])

In [43]:
16*1*2*2 + 16 + 10*2704 + 10 

In [44]:
27130/1628170

## C. 유명함

`-` <https://brunch.co.kr/@hvnpoet/109>

# 4. CNN 핵심레이어

## A. `torch.nn.ReLU`

**(예시1) 연산방법**

In [39]:
img = torch.randn(1,1,4,4) # (4,4) 흑백이미지 한장
relu = torch.nn.ReLU()

In [42]:
img

In [43]:
relu(img)

## B. `torch.nn.MaxPool2d`

**(예시1) 연산방법, kernel_size 의 의미**

In [57]:
img = torch.rand(1,1,4,4)
mp = torch.nn.MaxPool2d(kernel_size=2)

In [58]:
img

In [59]:
mp(img)

**(예시2) 이미지크기와 딱 맞지않는 커널일경우?**

In [63]:
img = torch.rand(1,1,5,5)
mp = torch.nn.MaxPool2d(kernel_size=3)

In [64]:
img

In [65]:
mp(img)

**(예시3) 정사각형이 아닌 커널**

In [69]:
img = torch.rand(1,1,4,4)
mp = torch.nn.MaxPool2d(kernel_size=(4,2))

In [70]:
img

In [72]:
mp(img)

## C. `torch.nn.Conv2d`

**(예시1) 연산방법, stride=2**

In [4]:
img = torch.rand(1,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=2)

In [5]:
img

In [6]:
conv(img)

??

In [7]:
conv.weight.data, conv.bias.data

In [8]:
(img[:,  :,  :2,  :2] * conv.weight.data).sum()+conv.bias.data, conv(img)

In [9]:
(img[:,  :,  :2,  2:] * conv.weight.data).sum()+conv.bias.data, conv(img)

In [11]:
(img[:,  :,  2:,  :2] * conv.weight.data).sum()+conv.bias.data, conv(img)

In [12]:
(img[:,  :,  2:,  2:] * conv.weight.data).sum()+conv.bias.data, conv(img)